__собираем данные__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# каталог с файлами объявлений
avito_raw_data_path = 'data/raw/land/avito' 
# cian_raw_data_path = 'data/raw/cian'

loc_file_path = 'data/location.pkl' # таблица адресов
result_file_path = 'data/data_land.pkl'

In [5]:
# ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
# ts

## собираем данные авито.ру

In [6]:
import re
from os import listdir

raw_data_files = sorted([
        avito_raw_data_path+'/'+f 
        for f in listdir(avito_raw_data_path) 
        if re.match(r'.+\.xlsx$',f)
    ])
raw_data_files

['data/raw/land/avito/avito_2022-09-08_16-05_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-12_13-51_raw.xlsx']

In [7]:
data_avito = pd.concat([
        pd
        .read_excel(f)
        #.dropna()
        .drop_duplicates(['avito_id','description'])
        .rename(columns={'page':'avito_page'})
        .reset_index(drop=True)  
        for f in raw_data_files
    ]).reset_index(drop=True)

print(len(data_avito))

2183


In [8]:
# data_avito

In [9]:
from lib.avito import AvitoDataCleanerRealtyLand

data_avito = AvitoDataCleanerRealtyLand().transform( data_avito )

print(len(data_avito))

2183


In [10]:
data_avito.sample(3)

,avito_id,title,price,obj_name,adr,description,avito_page,ts,area,is_IJS,priceM,priceMU,area_size_category
541,2314893650,Участок 15 сот. (ИЖС),4290000,,с. Широкое,"Продаётся земельный участок в селе Широкое ул. Приозерная для индивидуальной жилой застройки ( кол 2.1). Кадастровый Номер 91:01:049001:1032. Присвоен адрес: ул. Приозерная 26-А. Подойдет под небольшую базу отдыха, зелёный туризм, спортивный лагерь глемпин",12,2022-09-08 16:04:40.227,15.00,True,4.29,0.29,8-20
2115,2485390073,Участок 10 сот. (ИЖС),5300000,,"с. Широкое, ул. Клюшкина","Объект №242. \n\nВ продаже участок 10 соток Ижс с невероятным видом на горы в Байдарской долине (с. Широкое). \nУчасток правильной формы, с круглогодичным подъездом. До асфальтированной дороги 200м, там же остановка общественно транспорта. \nПо границе есть ст",22,2022-09-12 13:51:16.745,10.00,True,5.30,0.53,8-20
1365,1222596224,Участок 5 сот. (ИЖС),2000000,,мыс Лукулл,"Неразработанный участок на мысе Лукулл, готов к подключению воды, эл. Энергии. До моря 5 минут ходьбы. Комплект документов. Кадастровый номер 91:04:010001:189.",6,2022-09-12 13:51:16.745,5.00,True,2.00,0.40,4-8


In [11]:
assert len(data_avito)>0

In [12]:
data_avito.info(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2183 entries, 0 to 2182
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   avito_id            2183 non-null   int64         
 1   title               2183 non-null   object        
 2   price               2183 non-null   int64         
 3   obj_name            2183 non-null   object        
 4   adr                 2183 non-null   object        
 5   description         2183 non-null   object        
 6   avito_page          2183 non-null   int64         
 7   ts                  2183 non-null   datetime64[ns]
 8   area                2183 non-null   float64       
 9   is_IJS              2183 non-null   bool          
 10  priceM              2183 non-null   float64       
 11  priceMU             2183 non-null   float64       
 12  area_size_category  2183 non-null   category      
dtypes: bool(1), category(1), datetime64[ns](1), floa

In [13]:
## собираем данные циан.ру

In [14]:
# import re
# from os import listdir

# raw_data_files = sorted([
#         cian_raw_data_path+'/'+f 
#         for f in listdir(cian_raw_data_path) 
#         if re.match(r'.+\.pkl$',f)
#         #if re.match(r'.+\.xlsx$',f)
    
#     ])
# raw_data_files

In [15]:
# data_cian = pd.concat([
#         # pd.read_excel(f)
#         pd.read_pickle(f)
#          .dropna()
#          .drop_duplicates(['LinkArea','Description',])
#          .rename(columns={'page':'cian_page'})
#          .reset_index(drop=True)  
#         for f in raw_data_files
#     ]).reset_index(drop=True)

In [16]:
# data_cian.info(2)

In [17]:
# swap_title = data_cian['OfferSubtitle'].str.match(r'.*\d+ м², \d+/\d+ эт.*') 

# data_cian = pd.concat([
#         data_cian[ ~swap_title ],
#         data_cian[ swap_title ]
#             .rename(columns={'OfferTitle':'OfferSubtitle','OfferSubtitle':'OfferTitle'})
#     ]).reset_index(drop=True)


In [18]:
# data_cian

In [19]:
# from lib.cian import CianDataCleaner

# data_cian = CianDataCleaner().transform( data_cian )

# print(len(data_cian))

In [20]:
# assert len(data_cian)>0

In [21]:
# data_cian.info(2)

In [22]:
# data_cian
# data_cian.query('nrooms==0')
# data_cian[ data_cian['title'].str.len()<1 ]
# data_cian[ data_cian['obj_name'].str.len()>0 ][['title','obj_name',]] # .to_csv('tmp/cian_title.csv',sep='\t')|

----

In [23]:
# data = pd.concat([data_avito,data_cian]).reset_index(drop=True) 
# print(len(data))
# assert len(data)>0

---

In [24]:
data = data_avito.reset_index(drop=True) 
print(len(data))
assert len(data)>0

2183


### обновляем таблицу адресов

In [25]:
from lib.locator import LocationUpdater
from lib.locator import AddressTransformerSev

loc = LocationUpdater(
        address_transformer=AddressTransformerSev(),
    ).transform(
        adr=data['adr'],
        loc=pd.read_pickle(loc_file_path),
        show_pbar=True,
    )

[INFO    ] 2022-09-12 13:56:46 | LocationUpdater: 2703 addresses in location table
[INFO    ] 2022-09-12 13:56:46 | LocationUpdater: 3554 addresses total
[INFO    ] 2022-09-12 13:56:46 | LocationUpdater: 2703 addresses defined
[INFO    ] 2022-09-12 13:56:46 | LocationUpdater: 851 addresses undefined


  0%|          | 0/851 [00:00<?, ?it/s]

[INFO    ] 2022-09-12 14:08:24 | LocationUpdater: 446 new addresses found


In [26]:
loc.sample(3)

,adr,latitude,longitude,truncated
1026,"улица Ленина, 32",44.61,33.53,False
1017,"Нахимовский муниципальный округ, площадь Генерала Захарова, 6",44.63,33.54,False
3039,"Балаклава, Фонтанный пер., 18В",44.51,33.60,True


In [27]:
assert len(loc)>0
loc.to_pickle(loc_file_path)

### дополняем данные геометкой

In [28]:
data = data.merge(loc[['adr','latitude','longitude',]],on=['adr'],how='left')

print('всего записей:', len( data) )
print('записей без геометки:', len( data[ data['latitude'].isnull() ] ) )

всего записей: 2183
записей без геометки: 1000


In [29]:
assert len(data)>0
data.to_pickle(result_file_path)

In [30]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2183 entries, 0 to 2182
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   avito_id            2183 non-null   int64         
 1   title               2183 non-null   object        
 2   price               2183 non-null   int64         
 3   obj_name            2183 non-null   object        
 4   adr                 2183 non-null   object        
 5   description         2183 non-null   object        
 6   avito_page          2183 non-null   int64         
 7   ts                  2183 non-null   datetime64[ns]
 8   area                2183 non-null   float64       
 9   is_IJS              2183 non-null   bool          
 10  priceM              2183 non-null   float64       
 11  priceMU             2183 non-null   float64       
 12  area_size_category  2183 non-null   category      
 13  latitude            1183 non-null   float64     

In [31]:
# for f in raw_data_files:
#     ts = dtm.strptime( re.sub(r'.*/avito_','',f), '%Y-%m-%d_%H-%M_raw.xlsx')
#     df = pd.read_excel(f)
#     df['ts'] = ts
#     df.to_excel(f,index=False)
#     print(ts)

In [32]:
# class AvitoDataCleanerRealtyLand:
    
#     def transform(self,data):
#         df = data.copy()
        
#         symb_lat = 'yexapocEXAPOCTHKBM'
#         symb_rus = 'уехаросЕХАРОСТНКВМ'
#         l2r = str.maketrans(symb_lat,symb_rus) 
        
#         df['title'] = df['title'].str.extract( r'«(.*)»',expand=False).apply(lambda s: s.translate(l2r) )
        
#         area = df['title'].str.extract( r'(\d+,?\d*)\s*(сот|га)',expand=True)
#         area.columns=['area','area_unit']
#         df = pd.concat([df,area],axis=1)
                       
#         df['is_IJS'] = df['title'].str.lower().str.match(r'.*ижс.*')
#         df['obj_name'] = df['obj_name'].fillna('')
#         df['price'] = df['price'].astype(int)
#         df['priceM'] = df['price']/1e6
        
#         df['area'] = df['area'].fillna('0.').str.replace(',','.').astype(float)
        
#         df.loc[ df['area_unit']=='га', 'area' ] = df.query('area_unit=="га"')['area']*100.
#         df = df.drop(columns=['area_unit'])
        
#         return df

In [33]:
# data_avito['title'].str.extract( r'(\d+,?\d*)\s*(сот|га)',expand=True) #, columns=['a','b'])
        

In [34]:
# # import re
# # s='Участок 9,7 сот. (ИЖС)'
# s='Участок 7,8 сот. (ИЖС)'
# s='Участок 7,8 га. (ИЖС)'

# s='Участок 7,8 м. (ИЖС)'

# # s='Участок 5 сот. (ИЖС)'

# re.search( r'(\d+,?\d*)\s*(сот|га)',s) #.group(1)

# # # len(s)